In [ ]:
!pip install selenium bs4

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import time 
import csv
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch

In [2]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:1


# 1. Web Scraping

In [14]:
url = 'https://www.tripadvisor.com/Hotel_Review-g150782-d155753-Reviews-Krystal_Monterrey-Monterrey_Northern_Mexico.html#REVIEWS'

In [33]:
# open the file to save the review
csvFile = open("hotel_reviews2.csv", 'a', encoding="utf-8")
csvWriter = csv.writer(csvFile)

In [ ]:
pages_to_scrape=120

driver = webdriver.Firefox()
driver.get(url)

for i in range(0, pages_to_scrape):
    print(i)
   
    time.sleep(2)

    driver.find_element("xpath","/html/body/div[2]/div[2]/div[2]/div[10]/div/div[1]/div[1]/div/div/div[3]/div[1]/div[1]/div[4]/ul/li[2]/label").click()

    # Click en el enlace "expandir opinión" para revelar toda la opinión.
    driver.find_element("xpath",".//div[contains(@data-test-target, 'expand-review')]").click()

    # Contenedor que contendrá todas las opiniones de la página.
    container = driver.find_elements("xpath","//div[@data-reviewid]")
   
   # Opiniones dentro del contenedor
    for j in range(len(container)):
        
        # Obtener calificacion
        rating = container[j].find_element("xpath",".//span[contains(@class, 'ui_bubble_rating bubble_')]").get_attribute("class").split("_")[3]
        # Obtener titulo
        title = container[j].find_element("xpath",".//div[contains(@data-test-target, 'review-title')]").text
        # Obtener opinion
        review = container[j].find_element("xpath",".//span[@class='QewHA H4 _a']/span").text.replace("\n", "  ")
        # Guarda los datos en el csv y luego continúa procesando la siguiente opinión
        csvWriter.writerow([rating, title, review])

    # Cuando todas las opiniones en el contenedor hayan sido procesadas, cambia la página y repite
    driver.find_element("xpath",'.//a[@class="ui_button nav next primary "]').click()

# Cuando todas las páginas hayan sido procesadas, cierra el driver
driver.quit()


# 2. Problema binario

In [19]:
df = pd.read_csv("hotel_reviews2.csv", names=['rating', 'title', 'review'])
df.head()

,rating,title,review
0,50,Excelente Hotel en Centro de Monterrey,Si buscas un hotel en el Centro de Monterrey H...
1,50,Tus eventos en el Krystal,Muy buen servicio y atención del todo el perso...
2,50,Excelente opción para eventos y grupos.,Excelente servicio y atención personal. Muy bu...
3,50,Geting ready,El mes pasado mi prima tuvo su getting y me en...
4,50,Realizamos un evento y nuestros clientes se fu...,Excelente servicio y atención al cliente.Sobre...


In [20]:
df['rating'].value_counts()

rating
50    571
40    303
30    171
10     60
20     54
Name: count, dtype: int64

In [21]:
df['binary_rating'] = df['rating'].apply(lambda x: 1 if x >= 40 else 0)
df['binary_rating'].value_counts()

binary_rating
1    874
0    285
Name: count, dtype: int64

In [35]:
neg_comments_per = df['binary_rating'].value_counts()[0] / df['binary_rating'].value_counts().sum()
pos_comments_per = df['binary_rating'].value_counts()[1] / df['binary_rating'].value_counts().sum()
print("Negative comments percentage: ", neg_comments_per)
print("Positive comments percentage: ", pos_comments_per)

Negative comments percentage:  0.2459016393442623
Positive comments percentage:  0.7540983606557377



En los datos obtenidos solo había calificaciones con los siguientes valores:

| Rating | Count |
|--------|-------|
| 50     | 571   |
| 40     | 303   |
| 30     | 171   |
| 10     | 60    |
| 20     | 54    |

Para volverlo un problema de clasificación de sentimientos binarios, se optó por considerar los comentarios con calificación mayor o igual a 40 como positivos.

Tras la conversión, el ~24.59% de los comentarios obtenidos son negativos, mientras que el ~75.40% son positivos.


# 3. Modelo Transformer para generación de comentarios "sintéticos"

In [ ]:
!pip install einops

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

model = "tiiuae/falcon-7b"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map=1,
    max_length=400,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
sequences = pipeline(
   "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\nDaniel: Hello, Girafatron!\nGirafatron:",
    max_length=2048,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
The model 'RWForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusFor

Result: Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.
Daniel: Hello, Girafatron!
Girafatron: Hello, Daniel.
Daniel: I'm sorry to interrupt you, but I'm having a problem with my giraffe.
Girafatron: You have a problem with a giraffe? You have a problem with a giraffe?
*Girafatron gets up and looks for his gun*
Daniel: *looks nervous*
Girafatron: Where is it? Where is it?!
Daniel: It's in my house, in the attic.
Daniel: Oh, I see you have it.
Girafatron: I got you, you stupid giraffe! *pulls trigger on the gun and blows a hole in the wall of Daniel's house, killing him.*
Daniel: *dead* (I'm sorry, I was busy.)
Daniel: *dead* (I'm busy.)
Daniel: *dead*
Girafatron: I got you, you stupid giraffe! *pulls trigger on the gun and blows a hole in the wall of Daniel's house, killing him.*


In [ ]:
!pip install langchain openai

In [10]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain, HuggingFacePipeline
from langchain.llms import OpenAI

In [28]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs={
    'temperature':0.45, 
    'do_sample': True, 'num_return_sequences': 1, 'eos_token_id': tokenizer.eos_token_id
})

In [11]:
llm = OpenAI(model_name="gpt-3.5-turbo", openai_api_key="")

c:\Users\m1000\anaconda3\envs\tensorflow\lib\site-packages\langchain\llms\openai.py:748: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [25]:
template = """
Comentario negativo 1: Recién remodelado, El hotel acaba de ser remodelado pero se nota una pésima gestión. Las almohadas como estaban hace un año todavía no han sido cambiadas, son viejas bolsas de plumón apestosas aplastadas e incómodas. Hace un año me hospedé ahi y tuve el mismo problema increible no deberian comprar almohadas nuevas para todo el hotel ademas no tienen frazadas en las camas y el clima extremo de monterrey me hizo congelar por 3 noches el aire acondicionado solo enfria pero no tienen calefacción. El personal está desorganizado y sin autoridad para poner orden. El restaurante es viejo y necesita remodelación. \n
Comentario negativo 2: {review}\n
Comentario negativo 3:
"""
prompt = PromptTemplate(template=template, input_variables=["review"])
llm_chain = LLMChain(prompt=prompt, llm=llm)
random_review = df[df['binary_rating'] == 0]['review'].sample(1).values[0]
result = llm_chain.run(random_review)
print(result)

No recomiendo este hotel en absoluto. La habitación estaba sucia y maloliente, probablemente no habían limpiado a fondo en semanas. El baño estaba lleno de moho y el inodoro no funcionaba correctamente. Incluso encontré cucarachas en la habitación. El personal fue grosero y poco servicial. No esperes una experiencia agradable si decides alojarte aquí. Hay opciones mucho mejores en la zona.


In [ ]:
new_reviews = []

# Use tqdm in the for loop to display a progress bar
for i in tqdm(range(0,600)):
    random_review = df[df['binary_rating'] == 0]['review'].sample(1).values[0]
    result = llm_chain.run(random_review)
    print(result)
    new_reviews.append(result)


In [31]:
new_df = pd.DataFrame(new_reviews, columns=["review"])

# Save the DataFrame to a CSV file
new_df.to_csv("synthetic_reviews.csv", index=False)

In [7]:
new_df = pd.read_csv("synthetic_reviews.csv")
new_df

,review
0,La experiencia en este hotel fue decepcionante...
1,Este hotel es un completo desastre. La habitac...
2,"Pésimo servicio en el hotel, la gente de recep..."
3,La experiencia en el hotel fue terrible. Desde...
4,Mi experiencia en este hotel fue bastante dece...
...,...
595,El servicio al cliente fue muy pobre y decepci...
596,Mi experiencia en el Fiesta Americana fue muy ...
597,Mi experiencia en este hotel fue completamente...
598,La peor experiencia hotelera de mi vida. El pe...


In [8]:
new_df['binary_rating'] = 0
new_df

,review,binary_rating
0,La experiencia en este hotel fue decepcionante...,0
1,Este hotel es un completo desastre. La habitac...,0
2,"Pésimo servicio en el hotel, la gente de recep...",0
3,La experiencia en el hotel fue terrible. Desde...,0
4,Mi experiencia en este hotel fue bastante dece...,0
...,...,...
595,El servicio al cliente fue muy pobre y decepci...,0
596,Mi experiencia en el Fiesta Americana fue muy ...,0
597,Mi experiencia en este hotel fue completamente...,0
598,La peor experiencia hotelera de mi vida. El pe...,0


In [10]:
df.drop(columns=['rating','title'], inplace=True)

In [11]:
final_df = pd.concat([df, new_df], ignore_index=True)
final_df

,review,binary_rating
0,Si buscas un hotel en el Centro de Monterrey H...,1
1,Muy buen servicio y atención del todo el perso...,1
2,Excelente servicio y atención personal. Muy bu...,1
3,El mes pasado mi prima tuvo su getting y me en...,1
4,Excelente servicio y atención al cliente.Sobre...,1
...,...,...
1754,El servicio al cliente fue muy pobre y decepci...,0
1755,Mi experiencia en el Fiesta Americana fue muy ...,0
1756,Mi experiencia en este hotel fue completamente...,0
1757,La peor experiencia hotelera de mi vida. El pe...,0


In [12]:
neg_comments_per = final_df['binary_rating'].value_counts()[0] / final_df['binary_rating'].value_counts().sum()
print("Negative comments percentage: ", neg_comments_per)

Negative comments percentage:  0.503126776577601


In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "recall": recall, "precision": precision, "f1": f1}

In [6]:
final_df.to_csv("final_reviews.csv", index=False)

Los datos obtenidos se encontraban desbalanceados; contando con ~25% comentarios negativos y ~75% comentarios positivos. Por lo tanto, se optó por utilizar un modelo (LLM) para generar comentarios "sintéticos" negativos para balancear el porcentaje ambos tipos de comentarios.

Se consideró que los datos sintéticos generados son similares a los datos reales, por lo que esto podría ayudar al modelo a aprender de manera efectiva; esto se valida más adelante durante las etapas de entrenamiento (fine-tuning) del modelo. Asimismo, se toma en consideración que si los datos sintéticos no fuesen representativos, podrían haber llevado al modelo a aprender patrones incorrectos; afectando, así, el rendimiento del modelo. De esta manera, es importante monitorizar los resultados con, y sin, los datos sintéticos durante las etapas de entrenamiento y validación.

# 4. Partición de datos originales

In [37]:
X_train, X_test, y_train, y_test = train_test_split(df['review'].copy(), df['binary_rating'].copy(), test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# 5. Fine-tuning de Modelo de clasificación binaria en español (datos originales)

In [36]:
from transformers import DataCollatorWithPadding, AutoModelForSequenceClassification, AutoTokenizer, pipeline

model = AutoModelForSequenceClassification.from_pretrained("finiteautomata/beto-sentiment-analysis", num_labels=3).to(device)
tokenizer = AutoTokenizer.from_pretrained("finiteautomata/beto-sentiment-analysis")

# if tokenizer.pad_token is None:
#     tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# # Configura '[PAD]' como el token oficial de padding
# tokenizer.pad_token = '[PAD]'
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [38]:
import torch
from torch.utils.data import Dataset

class ReviewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Tokenización
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, max_length=512, return_tensors='pt')
val_encodings = tokenizer(X_val.tolist(), truncation=True, padding=True, max_length=512, return_tensors='pt')

# Conversión a conjuntos de datos de HuggingFace
train_dataset = ReviewsDataset(train_encodings, y_train.tolist())
val_dataset = ReviewsDataset(val_encodings, y_val.tolist())



In [41]:
def get_sentiment(pipeline, text, max_length=512):
    if len(text) > max_length:
      text = text[:max_length]
    return pipeline.predict(text)
sentiment_pipeline = pipeline('sentiment-analysis', tokenizer=tokenizer, model=model.cpu())
print(X_test.tolist()[0])
prediction = get_sentiment(sentiment_pipeline, X_test.tolist()[0])
prediction

De los mejores lugares, con el mejor servicio. Lugar cercano al centro de la ciudad, muchos lugares que visitar, puedes apoyarte en la gente del Hotel para recomendarte lugares. Cuenta con excelente restaurante, gimnasio y alberca.


[{'label': 'POS', 'score': 0.9961075186729431}]

In [42]:
from tqdm import tqdm

# Make predictions on the test set
predictions_default_model = [get_sentiment(sentiment_pipeline, review) for review in tqdm(X_test)]

# Convert the predictions to 1 or 0 sentiments
predictions_default_model = [1 if prediction[0]['label'] == 'POS' else 0 for prediction in predictions_default_model]

# Metrics
test_accuracy = accuracy_score(y_test, predictions_default_model)
test_precision = precision_score(y_test, predictions_default_model, average='weighted')
test_recall = recall_score(y_test, predictions_default_model, average='weighted')
test_f1_score = f1_score(y_test, predictions_default_model, average='weighted')

print(f'Accuracy of default model: {test_accuracy}')
print(f'Precision of default model: {test_precision}')
print(f'Recall of default model: {test_recall}')
print(f'F1 Score of default model: {test_f1_score}')


100%|██████████| 232/232 [00:21<00:00, 10.93it/s]

Accuracy of default model: 0.8663793103448276
Precision of default model: 0.8967991169977924
Recall of default model: 0.8663793103448276
F1 Score of default model: 0.8727061427780852


In [27]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

training_args = TrainingArguments(
   output_dir="./results",
   learning_rate=2e-5,
   per_device_train_batch_size=4,
   per_device_eval_batch_size=4,
   num_train_epochs=10,
   weight_decay=0.01,
   save_strategy="steps",
   evaluation_strategy="steps",
   eval_steps=500,
   load_best_model_at_end=True,
   logging_dir='./logs',            # Directorio de los logs
   logging_steps=10,
  #  push_to_hub=True,
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=train_dataset,
   eval_dataset=val_dataset,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
   callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

In [28]:
trainer.train()

c:\Users\m1000\anaconda3\envs\tensorflow\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/930 [00:00<?, ?it/s]

C:\Users\m1000\AppData\Local\Temp\ipykernel_43156\221027803.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
c:\Users\m1000\anaconda3\envs\tensorflow\lib\site-packages\torch\nn\parallel\_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
c:\Users\m1000\anaconda3\envs\tensorflow\lib\site-packages\torch\cuda\nccl.py:15: UserWarning: PyTorch is not compiled with NCCL support
  w

{'loss': 1.2264, 'learning_rate': 1.978494623655914e-05, 'epoch': 0.11}
{'loss': 0.6074, 'learning_rate': 1.956989247311828e-05, 'epoch': 0.22}
{'loss': 0.3976, 'learning_rate': 1.935483870967742e-05, 'epoch': 0.32}
{'loss': 0.3278, 'learning_rate': 1.913978494623656e-05, 'epoch': 0.43}
{'loss': 0.4197, 'learning_rate': 1.89247311827957e-05, 'epoch': 0.54}
{'loss': 0.0998, 'learning_rate': 1.870967741935484e-05, 'epoch': 0.65}
{'loss': 0.4815, 'learning_rate': 1.849462365591398e-05, 'epoch': 0.75}
{'loss': 0.3788, 'learning_rate': 1.827956989247312e-05, 'epoch': 0.86}
{'loss': 0.3402, 'learning_rate': 1.806451612903226e-05, 'epoch': 0.97}
{'loss': 0.0729, 'learning_rate': 1.78494623655914e-05, 'epoch': 1.08}
{'loss': 0.1035, 'learning_rate': 1.763440860215054e-05, 'epoch': 1.18}
{'loss': 0.0396, 'learning_rate': 1.741935483870968e-05, 'epoch': 1.29}
{'loss': 0.2807, 'learning_rate': 1.7204301075268818e-05, 'epoch': 1.4}
{'loss': 0.3315, 'learning_rate': 1.6989247311827958e-05, 'epoch':

  0%|          | 0/24 [00:00<?, ?it/s]

{'eval_loss': 0.628998875617981, 'eval_accuracy': 0.8978494623655914, 'eval_recall': 0.9444444444444444, 'eval_precision': 0.9251700680272109, 'eval_f1': 0.9347079037800687, 'eval_runtime': 2.6424, 'eval_samples_per_second': 70.39, 'eval_steps_per_second': 9.083, 'epoch': 5.38}


C:\Users\m1000\AppData\Local\Temp\ipykernel_43156\221027803.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
c:\Users\m1000\anaconda3\envs\tensorflow\lib\site-packages\torch\nn\parallel\_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
c:\Users\m1000\anaconda3\envs\tensorflow\lib\site-packages\torch\cuda\nccl.py:15: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')


{'loss': 0.0003, 'learning_rate': 9.03225806451613e-06, 'epoch': 5.48}
{'loss': 0.0002, 'learning_rate': 8.81720430107527e-06, 'epoch': 5.59}
{'loss': 0.0002, 'learning_rate': 8.602150537634409e-06, 'epoch': 5.7}
{'loss': 0.0008, 'learning_rate': 8.387096774193549e-06, 'epoch': 5.81}
{'loss': 0.0002, 'learning_rate': 8.172043010752689e-06, 'epoch': 5.91}
{'loss': 0.0003, 'learning_rate': 7.956989247311828e-06, 'epoch': 6.02}
{'loss': 0.0003, 'learning_rate': 7.741935483870968e-06, 'epoch': 6.13}
{'loss': 0.0023, 'learning_rate': 7.526881720430108e-06, 'epoch': 6.24}
{'loss': 0.0002, 'learning_rate': 7.311827956989248e-06, 'epoch': 6.34}
{'loss': 0.0002, 'learning_rate': 7.096774193548388e-06, 'epoch': 6.45}
{'loss': 0.0002, 'learning_rate': 6.881720430107528e-06, 'epoch': 6.56}
{'loss': 0.0002, 'learning_rate': 6.666666666666667e-06, 'epoch': 6.67}
{'loss': 0.0002, 'learning_rate': 6.451612903225806e-06, 'epoch': 6.77}
{'loss': 0.0002, 'learning_rate': 6.236559139784947e-06, 'epoch': 6

TrainOutput(global_step=930, training_loss=0.06690130372511684, metrics={'train_runtime': 372.2627, 'train_samples_per_second': 19.905, 'train_steps_per_second': 2.498, 'train_loss': 0.06690130372511684, 'epoch': 10.0})

In [29]:
trainer.evaluate()

C:\Users\m1000\AppData\Local\Temp\ipykernel_43156\221027803.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


  0%|          | 0/24 [00:00<?, ?it/s]

{'eval_loss': 0.628998875617981,
 'eval_accuracy': 0.8978494623655914,
 'eval_recall': 0.9444444444444444,
 'eval_precision': 0.9251700680272109,
 'eval_f1': 0.9347079037800687,
 'eval_runtime': 3.713,
 'eval_samples_per_second': 50.094,
 'eval_steps_per_second': 6.464,
 'epoch': 10.0}

A continuación se muestran los resultados obtenidos con el modelo por defecto, así como el modelo ajustado con los datos originales (sin datos sintéticos):

| Metric | Default Model | Fine-tuned Model |
|---|---|---|
| Accuracy  | 0.8664  | 0.8978  |
| Precision | 0.8968  | 0.9252  |
| Recall    | 0.8664  | 0.9444  |
| F1 Score  | 0.8727  | 0.9347  |

# 6. Fine-tuning de Modelo de clasificación binaria en español (con datos sintéticos)

In [43]:
final_df = pd.read_csv("final_reviews.csv")
final_df

,review,binary_rating
0,Si buscas un hotel en el Centro de Monterrey H...,1
1,Muy buen servicio y atención del todo el perso...,1
2,Excelente servicio y atención personal. Muy bu...,1
3,El mes pasado mi prima tuvo su getting y me en...,1
4,Excelente servicio y atención al cliente.Sobre...,1
...,...,...
1754,El servicio al cliente fue muy pobre y decepci...,0
1755,Mi experiencia en el Fiesta Americana fue muy ...,0
1756,Mi experiencia en este hotel fue completamente...,0
1757,La peor experiencia hotelera de mi vida. El pe...,0


In [30]:
X_train, X_test, y_train, y_test = train_test_split(final_df['review'].copy(), final_df['binary_rating'].copy(), test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [31]:
# Tokenización
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, max_length=512, return_tensors='pt')
val_encodings = tokenizer(X_val.tolist(), truncation=True, padding=True, max_length=512, return_tensors='pt')

# Conversión a conjuntos de datos de HuggingFace
train_dataset = ReviewsDataset(train_encodings, y_train.tolist())
val_dataset = ReviewsDataset(val_encodings, y_val.tolist())

In [32]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

training_args = TrainingArguments(
   output_dir="./results",
   learning_rate=2e-5,
   per_device_train_batch_size=4,
   per_device_eval_batch_size=4,
   num_train_epochs=10,
   weight_decay=0.01,
   save_strategy="steps",
   evaluation_strategy="steps",
   eval_steps=500,
   load_best_model_at_end=True,
   logging_dir='./logs',            
   logging_steps=10,
  #  push_to_hub=True,
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=train_dataset,
   eval_dataset=val_dataset,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
   callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

In [33]:
trainer.train()

c:\Users\m1000\anaconda3\envs\tensorflow\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1410 [00:00<?, ?it/s]

C:\Users\m1000\AppData\Local\Temp\ipykernel_43156\221027803.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
c:\Users\m1000\anaconda3\envs\tensorflow\lib\site-packages\torch\nn\parallel\_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
c:\Users\m1000\anaconda3\envs\tensorflow\lib\site-packages\torch\cuda\nccl.py:15: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')


{'loss': 0.2086, 'learning_rate': 1.9858156028368796e-05, 'epoch': 0.07}
{'loss': 0.2305, 'learning_rate': 1.971631205673759e-05, 'epoch': 0.14}
{'loss': 0.0803, 'learning_rate': 1.9574468085106384e-05, 'epoch': 0.21}
{'loss': 0.2565, 'learning_rate': 1.9432624113475178e-05, 'epoch': 0.28}
{'loss': 0.0568, 'learning_rate': 1.929078014184397e-05, 'epoch': 0.35}
{'loss': 0.135, 'learning_rate': 1.914893617021277e-05, 'epoch': 0.43}
{'loss': 0.1339, 'learning_rate': 1.9007092198581563e-05, 'epoch': 0.5}
{'loss': 0.0045, 'learning_rate': 1.8865248226950357e-05, 'epoch': 0.57}
{'loss': 0.0861, 'learning_rate': 1.872340425531915e-05, 'epoch': 0.64}
{'loss': 0.2625, 'learning_rate': 1.8581560283687945e-05, 'epoch': 0.71}
{'loss': 0.1516, 'learning_rate': 1.843971631205674e-05, 'epoch': 0.78}
{'loss': 0.0047, 'learning_rate': 1.8297872340425533e-05, 'epoch': 0.85}
{'loss': 0.0285, 'learning_rate': 1.8156028368794327e-05, 'epoch': 0.92}
{'loss': 0.0155, 'learning_rate': 1.801418439716312e-05, '

  0%|          | 0/36 [00:00<?, ?it/s]

...

  0%|          | 0/36 [00:00<?, ?it/s]

{'eval_loss': 0.12397931516170502, 'eval_accuracy': 0.9858156028368794, 'eval_recall': 0.9838709677419355, 'eval_precision': 0.9838709677419355, 'eval_f1': 0.9838709677419355, 'eval_runtime': 4.2315, 'eval_samples_per_second': 66.643, 'eval_steps_per_second': 8.508, 'epoch': 7.09}


C:\Users\m1000\AppData\Local\Temp\ipykernel_43156\221027803.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
c:\Users\m1000\anaconda3\envs\tensorflow\lib\site-packages\torch\nn\parallel\_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
c:\Users\m1000\anaconda3\envs\tensorflow\lib\site-packages\torch\cuda\nccl.py:15: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')


{'loss': 0.0001, 'learning_rate': 5.673758865248228e-06, 'epoch': 7.16}
{'loss': 0.0001, 'learning_rate': 5.531914893617022e-06, 'epoch': 7.23}
{'loss': 0.0, 'learning_rate': 5.390070921985816e-06, 'epoch': 7.3}
{'loss': 0.0001, 'learning_rate': 5.24822695035461e-06, 'epoch': 7.38}
{'loss': 0.0, 'learning_rate': 5.106382978723404e-06, 'epoch': 7.45}
{'loss': 0.0, 'learning_rate': 4.964539007092199e-06, 'epoch': 7.52}
{'loss': 0.0, 'learning_rate': 4.822695035460993e-06, 'epoch': 7.59}
{'loss': 0.0, 'learning_rate': 4.680851063829788e-06, 'epoch': 7.66}
{'loss': 0.0, 'learning_rate': 4.539007092198582e-06, 'epoch': 7.73}
{'loss': 0.0, 'learning_rate': 4.397163120567377e-06, 'epoch': 7.8}
{'loss': 0.0, 'learning_rate': 4.255319148936171e-06, 'epoch': 7.87}
{'loss': 0.0, 'learning_rate': 4.113475177304965e-06, 'epoch': 7.94}
{'loss': 0.0, 'learning_rate': 3.9716312056737595e-06, 'epoch': 8.01}
{'loss': 0.0, 'learning_rate': 3.8297872340425535e-06, 'epoch': 8.09}
{'loss': 0.0, 'learning_ra

TrainOutput(global_step=1410, training_loss=0.016444887505333988, metrics={'train_runtime': 816.9324, 'train_samples_per_second': 13.771, 'train_steps_per_second': 1.726, 'train_loss': 0.016444887505333988, 'epoch': 10.0})

In [34]:
trainer.evaluate()

C:\Users\m1000\AppData\Local\Temp\ipykernel_43156\221027803.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


  0%|          | 0/36 [00:00<?, ?it/s]

{'eval_loss': 0.09445394575595856,
 'eval_accuracy': 0.9893617021276596,
 'eval_recall': 0.9919354838709677,
 'eval_precision': 0.984,
 'eval_f1': 0.9879518072289156,
 'eval_runtime': 4.5855,
 'eval_samples_per_second': 61.499,
 'eval_steps_per_second': 7.851,
 'epoch': 10.0}

*¿Por qué en este caso no se justificaría utilizar la técnica out-of-box para entrenar y evaluar directamente los datos con los comentarios sintéticos?*

No se justificaría utilizar el modelo por defecto para evaluar el dataset con comentarios "sintéticos" ya que se busca que el modelo se ajuste a las características específicas del conjunto de datos utilizado; lo cual se logra con la técnica de "fine-tuning".

A continuación se muestran los resultados obtenidos:

| Metric        | Value                 |
|---------------|-----------------------|
| eval_accuracy | 0.9893617021276596    |
| eval_recall   | 0.9919354838709677    |
| eval_precision| 0.984                 |
| eval_f1       | 0.9879518072289156    |

# 7. Comparación de resultados

Los resultados obtenidos con datos sintéticos parecen ser significativamente mejores que los obtenidos con los datos originales. En todos los indicadores (precisión, recall, F1 y accuracy), el modelo entrenado con los datos sintéticos superó al modelo entrenado con los datos originales.

Este mejor rendimiento puede deberse a que los datos sintéticos ayudaron a balancear el conjunto de datos y a proporcionar más ejemplos para que el modelo aprendiera patrones más diversos. Y esta mejora en la representatividad de los datos podría haber permitido que el modelo generalizara mejor en el conjunto de validación.

En este caso, los comentarios sintéticos parecen haber sido útiles para mejorar el rendimiento del modelo. Sin embargo, resulta importante evaluar la calidad de los datos sintéticos y cómo afectan el rendimiento del modelo en cuestión, ya que pueden introducir ruido o desviaciones si no se generan correctamente.



| Metric       | Default Model | Fine-tuned Model with Original Data | Fine-tuned Model with Synthetic Data |
|--------------|-------------- |------------|-------------------------------------|
| Accuracy     | 0.8664        | 0.8978     | 0.9894                              |
| Precision    | 0.8968        | 0.9252     | 0.984                               |
| Recall       | 0.8664        | 0.9444     | 0.9919                              |
| F1 Score     | 0.8727        | 0.9347     | 0.9879                              | 

Esta tabla proporciona una vista directa de cómo cada modelo se compara en cada métrica. El modelo ajustado con datos sintéticos supera a los otros dos en todas las métricas.